In [ ]:
import numpy as np


In [ ]:
num_evaluations = 0
k_bisection = 0

In [ ]:
def initialize_population( num_individuals, num_variables ):
    """
    Khởi tạo quần thể gồm num_individuals cá thể. Mỗi cá thể có num_parameters biến.
    
    Arguments:
    num_individuals -- Số lượng cá thể
    num_variables -- Số lượng biến
    
    Returns:
    pop -- Ma trận (num_individuals, num_variables ) chứa quần thể mới được khởi tạo ngẫu nhiên.
    """
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ###
    pop = np.random.randint(2, size=(num_individuals, num_variables))
    
    ### DỪNG CODE TẠI ĐÂY ###
    
    return pop

In [ ]:
def onemax( ind ):
    """
    Hàm đánh giá OneMax: Đếm số bit 1 trong chuỗi nhị phân (cá thể ind).
    
    Arguments:
    ind -- Cá thể cần được đánh giá.

    Returns:
    value -- Giá trị của cá thể ind.
    """
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ###
    global num_evaluations
    num_evaluations += 1   
    value = np.sum(ind)
    
    ### DỪNG CODE TẠI ĐÂY ###
    
    return value

In [ ]:
def trap_5( ind ):
  global num_evaluations 
  num_evaluations += 1
  value = 0
  for i in range(0, len(ind), 5):
    sum = np.sum(ind[i:i+5])
    if (sum == 5):
      value += 5
    else:
      value += (4 - sum) 
  return value

In [ ]:
def pop_fitness_evaluator(pop, method):
  return np.array([method(ind) for ind in pop])

In [ ]:
def tournament_selection(pool, pool_fitness, tournament_size=4):
  num_individuals = len(pool)

  result = []

  for _ in range(2):
    indicies = np.arange(num_individuals)
    np.random.shuffle(indicies)
    
    for i in range(0, num_individuals, tournament_size):
      current_idx = list(indicies[i:i+tournament_size])

      fitness_points = []
      for idx in current_idx:
        fitness_points.append(pool_fitness[idx])
      
      best_fit_idx = current_idx[fitness_points.index(max(fitness_points))]

      result.append(best_fit_idx)
  
  return result

In [ ]:
def crossover( pop, method ):
    """
    Hàm biến đổi tạo ra các cá thể con.
    
    Arguments:
    pop -- Quàn thể hiện tại.

    Returns:
    offspring -- Quần thể chứa các cá thể con được sinh ra.
    """  
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ### 
    num_individuals = len(pop)
    num_parameters = len(pop[0])
    indices = np.arange(num_individuals)
    # Đảo ngẫu nhiên thứ tự các cá thể trong quần thể
    np.random.shuffle(indices)
    offspring = []
    
    for i in range(0, num_individuals, 2):
        idx1 = indices[i]
        idx2 = indices[i+1]
        offspring1 = list(pop[idx1])
        offspring2 = list(pop[idx2])
        
        if method == "UX":
          # Cài đặt phép lai đồng nhất uniform crossover. 
          for idx in range(0, num_parameters):
              r = np.random.rand()
              if r < 0.5:
                temp = offspring2[idx] 
                offspring2[idx] = offspring1[idx]
                offspring1[idx] = temp

        elif (method == "1X"):
          # Cài đặt phép lai một điểm 1X crossover.
          r = np.random.randint(num_parameters)
          for idx in range(r, num_parameters):
                temp = offspring2[idx] 
                offspring2[idx] = offspring1[idx]
                offspring1[idx] = temp


        offspring.append(offspring1)
        offspring.append(offspring2)


    ### DỪNG CODE TẠI ĐÂY ###
    
    offspring = np.array(offspring)
    return offspring

In [ ]:
def check_convergence(pop):
  num_individuals = len(pop)
  num_parameters = len(pop[0])
  for i in range(num_individuals - 1):
    for j in range(num_parameters):
      if pop[i,j] != pop[i+1,j]:
        return False
  return True

In [ ]:
def genetic_algorithm(num_individuals, num_parameters, fitness_func, crossover_type, random_seed):
    np.random.seed(random_seed)
    pop = initialize_population(num_individuals, num_parameters)
    pop_fitness = pop_fitness_evaluator(pop, fitness_func)

    while True:
        offspring = crossover(pop, crossover_type)
        offspring_fitness = pop_fitness_evaluator(offspring, fitness_func)

        pool = np.vstack([pop, offspring])
        pool_fitness = np.concatenate((pop_fitness, offspring_fitness))

        selected_indicies = tournament_selection(pool, pool_fitness)
        selection_set = pool[selected_indicies]
        selection_fitness = pool_fitness[selected_indicies]

        pop = np.copy(selection_set)
        pop_fitness = np.copy(selection_fitness)
        if check_convergence(pop):
          break

    # Return true if all individuals converge to best fitness otherwise false
    return np.all(pop_fitness == num_parameters)

In [ ]:
def run_10_sGA(num_individuals, num_parameters, fitness_func, crossover_type):
  global k_bisection
  MSSV = 19521432
  random_seed = MSSV + k_bisection * 10
  for i in range(10):
    if not (genetic_algorithm(num_individuals, num_parameters, fitness_func, crossover_type, random_seed + i)):
      return False
  return True    

In [ ]:
def bisection(num_parameters, fitness_func, crossover_type):
  # Find upper bound of MRPS
  n_upper = 4
  success = False
  while not (success):
    if n_upper == 8192:
      return 0
    n_upper *= 2
    success = run_10_sGA(n_upper, num_parameters, fitness_func, crossover_type)

  # Find MRPS
  n_lower = n_upper // 2
  while (n_upper - n_lower) / n_upper > 0.1:
    n = (n_upper + n_lower) // 2
    success = run_10_sGA(n, num_parameters, fitness_func, crossover_type)

    if success:
      n_upper = n
    else:
      n_lower = n 

    if (n_upper - n_lower) <= 2:
      break
  global num_evaluations
  num_evaluations = 0
  run_10_sGA(n_upper, num_parameters, fitness_func, crossover_type)
  return n_upper

In [ ]:
def run_10_bisection(num_parameters, fitness_func, crossover_type):
  # Return (mean of MRPS, mean of numbers of evaluations) of 10 successful bisections on a problem size.
  print("problem_size = {}; fitness_func = {}; crossover_method = {}".format(num_parameters, fitness_func.__name__, crossover_type))
  print("# MRPS  # num_evaluations  # random_seed")
  global num_evaluations
  global k_bisection 
  MRPS_list = []
  num_evaluations_list = []
  for k_bisection in range(10):
    MRPS = bisection(num_parameters, fitness_func, crossover_type)
    if MRPS == 0:
      num_evaluations = 0
    else:
      num_evaluations /= 10
    MRPS_list.append(MRPS)
    num_evaluations_list.append(num_evaluations)
    print("{} {} {}".format(MRPS, num_evaluations, 19521432 + k_bisection * 10))

  print("MRPS of 10 successful bisections: {}".format(MRPS_list))
  print("num_evaluations of 10 successful bisections: {}".format(num_evaluations_list))

  print("Mean and std of MRPS: {:.3f} {:.3f}".format(np.mean(MRPS_list), np.std(MRPS_list)))
  print("Mean and std of num_evaluations: {:.3f} {:.3f}".format(np.mean(num_evaluations_list), np.std(num_evaluations_list)))

In [ ]:
# Sample output
run_10_bisection(10, onemax, "UX")

problem_size = 10; fitness_func = onemax; crossover_method = UX
# MRPS  # num_evaluations  # random_seed
16 116.8 19521432
16 112.0 19521442
18 124.2 19521452
18 136.8 19521462
28 196.0 19521472
22 167.2 19521482
24 160.8 19521492
22 162.8 19521502
16 128.0 19521512
16 120.0 19521522
MRPS of 10 successful bisections: [16, 16, 18, 18, 28, 22, 24, 22, 16, 16]
num_evaluations of 10 successful bisections: [116.8, 112.0, 124.2, 136.8, 196.0, 167.2, 160.8, 162.8, 128.0, 120.0]
Mean and std of MRPS: 19.600 3.980
Mean and std of num_evaluations: 142.460 26.256


In [ ]:
run_10_bisection(10, trap_5, "1X")

problem_size = 10; fitness_func = trap_5; crossover_method = 1X
# MRPS  # num_evaluations  # random_seed
128 1164.8 19521432
136 1047.2 19521442
136 1183.2 19521452
104 936.0 19521462
120 1056.0 19521472
136 1237.6 19521482
136 1088.0 19521492
136 1196.8 19521502
88 809.6 19521512
128 1113.6 19521522
MRPS of 10 successful bisections: [128, 136, 136, 104, 120, 136, 136, 136, 88, 128]
num_evaluations of 10 successful bisections: [1164.8, 1047.2, 1183.2, 936.0, 1056.0, 1237.6, 1088.0, 1196.8, 809.6, 1113.6]
Mean and std of MRPS: 124.800 15.677
Mean and std of num_evaluations: 1083.280 123.657
